# Pipeline Bronze - E-commerce Brasileiro

Este notebook realiza a ingestão, transformação e carga dos dados do e-commerce brasileiro, utilizando a camada Bronze para garantir qualidade e rastreabilidade dos dados.

---

In [0]:
# criação da sessão do Spark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("BronzeLayer").getOrCreate()

## 1. Ingestão dos Dados CSV

Para realizar o Load dos dados na camada Bronze, os arquivos CSV no Volume são lidos, enriquecidos com o timestamp de ingestão e salvos nas tabelas Bronze correspondentes:
* `bronze.ft_consumidores`
* `bronze.ft_geolocalizacao`
* `bronze.ft_itens_pedidos`
* `bronze.ft_pagamentos_pedidos`
* `bronze.ft_avaliacoes_pedidos`
* `bronze.ft_pedidos`
* `bronze.ft_produtos`
* `bronze.ft_vendedores`
* `bronze.dm_categoria_produtos_traducao`

---

In [0]:
# caminho do Volume do ecommerce
path_files = "/Volumes/workspace/bronze/brazillian_ecommerce/"

# mapeamento do arquivo para a tabela com o nome padronizado
mapeamento_tabelas_bronze = {
    "olist_customers_dataset.csv": "bronze.ft_consumidores",
    "olist_geolocation_dataset.csv": "bronze.ft_geolocalizacao",
    "olist_order_items_dataset.csv": "bronze.ft_itens_pedidos",
    "olist_order_payments_dataset.csv": "bronze.ft_pagamentos_pedidos",
    "olist_order_reviews_dataset.csv": "bronze.ft_avaliacoes_pedidos",
    "olist_orders_dataset.csv": "bronze.ft_pedidos",
    "olist_products_dataset.csv": "bronze.ft_produtos",
    "olist_sellers_dataset.csv": "bronze.ft_vendedores",
    "product_category_name_translation.csv": "bronze.dm_categoria_produtos_traducao"
}

In [0]:
from pyspark.sql.functions import current_timestamp

# loop para carregar os arquivos
for key, value in mapeamento_tabelas_bronze.items():
    # leitura do arquivo
    df = (
        spark.read
        .format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(f"{path_files}/{key}")
    )

    # adição da coluna ingestion_timestamp, para armazenar a data de ingestão dos dados
    df_bronze = df.withColumn(
        "ingestion_timestamp", current_timestamp()
    )

    print(f"Tabela {key}:")

    # verificação do schema dos dados
    print("Esquema:")
    df_bronze.printSchema() 

    print("Dados (5 primeiras linhas):")
    display(df_bronze.limit(5))
    
    print(f"Salvando dados na tabela {value}...")
    (
        df_bronze.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(f"workspace.{value}")
    )

    print("-" * 60)

Tabela olist_customers_dataset.csv:
Esquema:
root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)

Dados (5 primeiras linhas):


customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,ingestion_timestamp
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,2025-11-20T00:11:24.602Z
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,2025-11-20T00:11:24.602Z
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,2025-11-20T00:11:24.602Z
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,2025-11-20T00:11:24.602Z
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,2025-11-20T00:11:24.602Z


Salvando dados na tabela bronze.ft_consumidores...
------------------------------------------------------------
Tabela olist_geolocation_dataset.csv:
Esquema:
root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)

Dados (5 primeiras linhas):


geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,ingestion_timestamp
1037,-23.54562128115268,-46.63929204800168,sao paulo,SP,2025-11-20T00:11:28.681Z
1046,-23.546081127035535,-46.64482029837157,sao paulo,SP,2025-11-20T00:11:28.681Z
1046,-23.54612896641469,-46.64295148361138,sao paulo,SP,2025-11-20T00:11:28.681Z
1041,-23.5443921648681,-46.63949930627844,sao paulo,SP,2025-11-20T00:11:28.681Z
1035,-23.541577961711493,-46.64160722329613,sao paulo,SP,2025-11-20T00:11:28.681Z


Salvando dados na tabela bronze.ft_geolocalizacao...
------------------------------------------------------------
Tabela olist_order_items_dataset.csv:
Esquema:
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)

Dados (5 primeiras linhas):


...

Salvando dados na tabela bronze.ft_produtos...
------------------------------------------------------------
Tabela olist_sellers_dataset.csv:
Esquema:
root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)

Dados (5 primeiras linhas):


seller_id,seller_zip_code_prefix,seller_city,seller_state,ingestion_timestamp
3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,2025-11-20T00:11:55.265Z
d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,2025-11-20T00:11:55.265Z
ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,2025-11-20T00:11:55.265Z
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,2025-11-20T00:11:55.265Z
51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,2025-11-20T00:11:55.265Z


Salvando dados na tabela bronze.ft_vendedores...
------------------------------------------------------------
Tabela product_category_name_translation.csv:
Esquema:
root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = false)

Dados (5 primeiras linhas):


product_category_name,product_category_name_english,ingestion_timestamp
beleza_saude,health_beauty,2025-11-20T00:11:58.175Z
informatica_acessorios,computers_accessories,2025-11-20T00:11:58.175Z
automotivo,auto,2025-11-20T00:11:58.175Z
cama_mesa_banho,bed_bath_table,2025-11-20T00:11:58.175Z
moveis_decoracao,furniture_decor,2025-11-20T00:11:58.175Z


Salvando dados na tabela bronze.dm_categoria_produtos_traducao...
------------------------------------------------------------


## 2. Extração da Cotação do Dólar

Além dos dados das planilhas, também é necessário extrair os dados de cotação do dólar, que são extraídos da API do Banco Central do Brasil, ano a ano, conforme o período informado.

---

### 2.1. Parâmetros de Execução

Primeiramente, é preciso definir os parâmetros necessários para trabalhar com a API. 

Sobre os períodos de busca dos dados, optei por utilizar via os widgets abaixo:
* **Data Início:** `data_inicio_formatada`
* **Data Fim:** `data_fim_formatada`

Os widgets servem como uma forma de alterar variáveis facilmente sem alterar código, tornando o notebook reutilizável.

Além disso, para armazenar todos os dados da API e posteriormente salvar no schema, criei a variável `todos_os_dados`. 

Também criei a variável `anos_para_buscar` para facilitar as requisições contínuas e garantir que todos os dados históricos da API estejam presentes.

---

In [0]:
from datetime import datetime

anos = [str(ano) for ano in range(1984, datetime.now().year + 1)]
dbutils.widgets.dropdown("data_inicio_ano", str(datetime.now().year), anos, "Ano Início")
data_inicio_historico = int(dbutils.widgets.get("data_inicio_ano"))

ano_atual = datetime.now().year

anos_slice = [str(ano) for ano in range(data_inicio_historico, ano_atual + 1)]
dbutils.widgets.dropdown("data_fim_ano", str(datetime.now().year), anos_slice, "Ano Fim")
data_fim_historico = int(dbutils.widgets.get("data_fim_ano"))

anos_para_buscar = range(data_inicio_historico, data_fim_historico + 1)

print(f"Buscando dados de {data_inicio_historico} até {data_fim_historico}...")
todos_os_dados = []

Buscando dados de 1984 até 2025...


### 2.2. Transformação e Carga da Cotação

Os dados extraídos são convertidos para DataFrame Spark, enriquecidos com o timestamp de ingestão e salvos na tabela `bronze.dm_cotacao_dolar`.

Abaixo, as últimas 10 cotações salvas:

---

In [0]:
import requests
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

for ano in anos_para_buscar:
    # definindo o intervalo de datas
    # início -> 1º de janeiro
    data_inicio_loop = f"01-01-{ano}"

    # fim -> 31 de dezembro
    data_fim_loop = f"12-31-{ano}"

    endpoint_url = (
        "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo("
        f"dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
        f"@dataInicial='{data_inicio_loop}'&@dataFinalCotacao='{data_fim_loop}'&"
        "$select=dataHoraCotacao,cotacaoCompra&$format=json"
    )

    print(f"Buscando dados para: {data_inicio_loop} até {data_fim_loop}...")
    
    try:
        # buscando os dados
        response = requests.get(endpoint_url)

        # checando status da requisição
        response.raise_for_status()  

        # pegando os dados da requisição
        data = response.json()
        
        # adiciona todos os dados da requisição no "dataframe"
        if 'value' in data and data['value']:
            todos_os_dados.extend(data['value'])
            
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar dados para o ano {ano}: {e}")

if len(todos_os_dados) > 0:
    print(f"\nBusca finalizada. Total de {len(todos_os_dados)} registros de cotação encontrados.")

    print(f"Exibindo 5 primeiras linhas:")
    display(todos_os_dados[:5])
else:
    print("Nenhum dado encontrado")

Buscando dados para: 01-01-1984 até 12-31-1984...
Buscando dados para: 01-01-1985 até 12-31-1985...
Buscando dados para: 01-01-1986 até 12-31-1986...
Buscando dados para: 01-01-1987 até 12-31-1987...
Buscando dados para: 01-01-1988 até 12-31-1988...
Buscando dados para: 01-01-1989 até 12-31-1989...
Buscando dados para: 01-01-1990 até 12-31-1990...
Buscando dados para: 01-01-1991 até 12-31-1991...
Buscando dados para: 01-01-1992 até 12-31-1992...
Buscando dados para: 01-01-1993 até 12-31-1993...
Buscando dados para: 01-01-1994 até 12-31-1994...
Buscando dados para: 01-01-1995 até 12-31-1995...
Buscando dados para: 01-01-1996 até 12-31-1996...
Buscando dados para: 01-01-1997 até 12-31-1997...
Buscando dados para: 01-01-1998 até 12-31-1998...
Buscando dados para: 01-01-1999 até 12-31-1999...
Buscando dados para: 01-01-2000 até 12-31-2000...
Buscando dados para: 01-01-2001 até 12-31-2001...
Buscando dados para: 01-01-2002 até 12-31-2002...
Buscando dados para: 01-01-2003 até 12-31-2003...


cotacaoCompra,dataHoraCotacao
2814.0,1984-12-03 11:29:00.0
2814.0,1984-12-03 16:38:00.0
2867.0,1984-12-04 11:17:00.0
2867.0,1984-12-05 11:31:00.0
2867.0,1984-12-05 12:40:00.0


In [0]:
# se há dados para salvar (sinal de que a API retornou dados)
if todos_os_dados:

    # definindo o schema para garantir os tipos corretos
    schema = StructType([
        StructField("dataHoraCotacao", StringType(), True),
        StructField("cotacaoCompra", DoubleType(), True)
    ])

    # criando o DataFrame a partir da lista completa
    df_cotacao_raw = spark.createDataFrame(todos_os_dados, schema=schema)
    
    # adicionando o timestamp de ingestão
    df_cotacao_bronze = df_cotacao_raw.withColumn(
        "ingestion_timestamp", current_timestamp()
    )
    
    # salvando na camada beonze
    table_name = "bronze.dm_cotacao_dolar"
    (
        df_cotacao_bronze.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(table_name)
    )
    
    print(f"Tabela {table_name} salva com sucesso.")

    # exibindo os 10 últimos registros
    display(df_cotacao_bronze.orderBy(col("dataHoraCotacao").desc()).limit(10))

Tabela bronze.dm_cotacao_dolar salva com sucesso.


dataHoraCotacao,cotacaoCompra,ingestion_timestamp
2025-11-19 13:09:27.626,5.334,2025-11-20T00:12:27.180Z
2025-11-18 13:10:26.459,5.3355,2025-11-20T00:12:27.180Z
2025-11-17 13:07:28.766,5.3102,2025-11-20T00:12:27.180Z
2025-11-14 13:11:28.202,5.2946,2025-11-20T00:12:27.180Z
2025-11-13 13:07:27.563,5.2814,2025-11-20T00:12:27.180Z
2025-11-12 13:11:25.169,5.2879,2025-11-20T00:12:27.180Z
2025-11-11 13:07:30.004,5.2723,2025-11-20T00:12:27.180Z
2025-11-10 13:03:31.138,5.3175,2025-11-20T00:12:27.180Z
2025-11-07 13:10:31.908,5.3561,2025-11-20T00:12:27.180Z
2025-11-06 13:02:49.425,5.3445,2025-11-20T00:12:27.180Z
